<a href="https://colab.research.google.com/github/nkthiebaut/algorithms/blob/master/4-np-complete-problems/week1-all_pairs_shortest_paths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithms 4 week 1: All pairs shortest paths
https://www.coursera.org/learn/algorithms-npcomplete/exam/cnDtw/programming-assignment-1/attempt

In [52]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import files
    uploaded = files.upload()
    data = uploaded['g1.txt'].decode('ascii')
else:
    uploaded_edges = open('./g2.txt', 'r')
    data = uploaded_edges.read()

Saving g1.txt to g1 (1).txt


In [53]:
import pandas as pd

n = int(data.split('\n')[0].split()[0])
m = int(data.split('\n')[0].split()[1])

costs = []
in_edges = []
out_edges = []
for line in data.split('\n')[1:-1]:
    in_edges.append(int(line.split()[0])-1)
    out_edges.append(int(line.split()[1])-1)
    costs.append(int(line.split()[2]))

df = pd.DataFrame({"source": in_edges, "target": out_edges, "cost": costs})

# Drop parallel edges with larger costs (does not affect shortest paths)
df = df.groupby(["source", "target"], as_index=False)["cost"].min()

assert max(df['source'].max(), df['target'].max())+1 == n

df.head()

,source,target,cost
0,0,13,6
1,0,24,47
2,0,69,22
3,0,81,31
4,0,97,17


In [54]:
import numpy as np
import time

def run_floyd_warshall(df):
    """ Run the Floyd-Marshall algorithm and return the shortest paths
    matrix. Diagonal elements indicate the presence of negative-cost
    cycles, if at least one of them is negative. 
    """
    n = max(df['source'].max(), df['target'].max()) + 1

    print("Initialization... ", end="")
    t0 = time.time()
    # A[i, j, 0] = 0 if i=j, cost(i, j) if (i, j) in E, infinity otherwise
    A = np.full([n, n], 1e6)

    for i in range(n):
        A[i, i] = 0

    for _, r in df.iterrows():
        A[r["source"], r["target"]] = r["cost"]
    print(f"completed after {time.time()-t0:.1f}s")
    t0 = time.time()
    
    # Main loop
    for k in range(n):
        if (k % 100 == 0) & (k > 0):
            print(f"k={k} after {time.time()-t0:.1f}s")
        # Vectorization trick: new_A[i, j] = A[i, k]+A[k, j]
        new_A = np.tile(A[:, k], (n, 1)).T + np.tile(A[k, :], (n, 1))
        A = np.minimum(A, new_A)  # vectorized minimum
    # Non-vectorized version (very slow)
#         for i in range(n):
#             for j in range(n):
#                 A[i, j] = min(A[i, j], A[i, k]+A[k, j])
    print(f"Completed after {time.time()-t0:.1f}s")
    return A[:, :]

shortest_paths = run_floyd_warshall(df)

Initialization...
Initialization completed after 4.6s
k=100 after 0.8s
k=200 after 1.5s
k=300 after 2.3s
k=400 after 3.1s
k=500 after 3.8s
k=600 after 4.6s
k=700 after 5.3s
k=800 after 6.1s
k=900 after 6.9s
Completed after 7.7s


In [56]:
# Check for negative-cost loop
end_diagonal = np.diagonal(shortest_paths)
negative_loop = (end_diagonal.min() < 0).any()
if negative_loop:
    print("Negative-cost loop!")
else:
    non_diagonal_elements = np.where(~np.eye(shortest_paths.shape[0], dtype=bool))
    print(f"Shortest shortest path length={shortest_paths[non_diagonal_elements].min()}")

Negative-cost loop!
